In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("Stock_RET.csv")

In [2]:
df.dtypes
# RET and ME is not the correct data types
# need to change the date to datetime object

PERMNO      int64
date        int64
EXCHCD      int64
RET        object
ME        float64
dtype: object

In [3]:
df.head(2)

,PERMNO,date,EXCHCD,RET,ME
0,10001,20160129,2,0.116779,87.4016
1,10001,20160229,2,-0.055288,82.5693


In [4]:
# check what's the meaning of offsets
df.date = pd.to_datetime(df.date, format='%Y%m%d', errors='ignore') + pd.offsets.MonthEnd(0)
df.EXCHCD = pd.to_numeric(df.EXCHCD, errors='coerce')
df.RET = pd.to_numeric(df.RET, errors='coerce')
df.ME = pd.to_numeric(df.ME, errors='coerce')

df = df.set_index(["date", "PERMNO"])
df.head()

,,EXCHCD,RET,ME
date,PERMNO,,,
2016-01-31,10001,2,0.116779,87.40160
2016-02-29,10001,2,-0.055288,82.56930
2016-03-31,10001,2,-0.006361,82.06748
2016-04-30,10001,2,-0.055698,76.73760
2016-05-31,10001,2,-0.021918,75.05568


## import Book to market value 資料

In [5]:
egg = pd.read_csv("Stock_BM.csv") # the time format is not the same as prev data
egg.head()

,PERMNO,yyyymm,BM
0,10001,201601,0.925455
1,10001,201602,0.925455
2,10001,201603,0.925455
3,10001,201604,0.925455
4,10001,201605,0.925455


In [6]:
egg['date'] = pd.to_datetime(egg.yyyymm, format='%Y%m', errors='ignore')
egg.head() # 新增的 date column 會是月初，改成月末的話要用 pandas offsets

,PERMNO,yyyymm,BM,date
0,10001,201601,0.925455,2016-01-01
1,10001,201602,0.925455,2016-02-01
2,10001,201603,0.925455,2016-03-01
3,10001,201604,0.925455,2016-04-01
4,10001,201605,0.925455,2016-05-01


In [7]:
bm = pd.read_csv("Stock_BM.csv")
bm['date'] = pd.to_datetime(bm.yyyymm, format='%Y%m', errors='ignore') + pd.offsets.MonthEnd(0)
bm = bm.drop('yyyymm', axis=1).set_index(["date", "PERMNO"])
bm.head()


,,BM
date,PERMNO,
2016-01-31,10001,0.925455
2016-02-29,10001,0.925455
2016-03-31,10001,0.925455
2016-04-30,10001,0.925455
2016-05-31,10001,0.925455


In [8]:
df = df.join(bm, how='inner') # inner: 取交集
df = df.sort_index(level=1)
df.head()

,,EXCHCD,RET,ME,BM
date,PERMNO,,,,
2016-01-31,10001,2,0.116779,87.40160,0.925455
2016-02-29,10001,2,-0.055288,82.56930,0.925455
2016-03-31,10001,2,-0.006361,82.06748,0.925455
2016-04-30,10001,2,-0.055698,76.73760,0.925455
2016-05-31,10001,2,-0.021918,75.05568,0.925455


In [11]:
# 分類的準則是 NYSE 的 data (作者說的)
df_nyse = df[df.EXCHCD==1]
df = df.drop(['EXCHCD'], axis=1)

df_nyse = df_nyse.drop(['EXCHCD'], axis=1)
df_nyse.head()
print(df_nyse.shape)

AttributeError: 'DataFrame' object has no attribute 'EXCHCD'

In [13]:
df_nyse.columns

Index(['RET', 'ME', 'BM'], dtype='object')

In [14]:
df_nyse.index

MultiIndex([('2016-01-31', 10051),
            ('2019-06-30', 10051),
            ('2019-07-31', 10051),
            ('2019-08-31', 10051),
            ('2019-09-30', 10051),
            ('2019-10-31', 10051),
            ('2019-11-30', 10051),
            ('2019-12-31', 10051),
            ('2020-01-31', 10051),
            ('2020-02-29', 10051),
            ...
            ('2020-03-31', 93426),
            ('2020-04-30', 93426),
            ('2020-05-31', 93426),
            ('2020-06-30', 93426),
            ('2020-07-31', 93426),
            ('2020-08-31', 93426),
            ('2020-09-30', 93426),
            ('2020-10-31', 93426),
            ('2020-11-30', 93426),
            ('2020-12-31', 93426)],
           names=['date', 'PERMNO'], length=69196)

In [12]:
me_nyse = df_nyse.unstack().xs('ME', axis=1) # xs: cross-sectional 
me_nyse = me_nyse.loc[me_nyse.index.month==6] # only rebalance on June
me_nyse.head()

PERMNO,10051,10104,10145,10147,10158,10220,10375,10516,10517,10606,...,93330,93372,93373,93374,93384,93418,93420,93422,93423,93426
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,NaN,168743.3389,88649.2168,53108.30239,125.78171,3713.39101,2163.79515,25201.39198,1592.49750,1622.89056,...,2692.91304,578.97646,1147.62492,4407.10650,285.85974,248.67304,1685.40300,4226.75724,5405.28625,163.09326
2017-06-30,NaN,207413.2355,101612.0244,NaN,211.17250,4828.54125,2729.42214,23518.61266,2748.67400,1761.95280,...,3446.77650,743.24390,531.86625,4939.58570,278.73907,NaN,1911.33565,2429.09040,5401.26210,210.92160
2018-06-30,NaN,175409.6893,107595.2665,NaN,354.79200,6212.05760,4142.38886,25627.40272,3056.49025,2181.08800,...,4388.77440,1089.64800,681.89460,5769.10740,80.47963,NaN,4115.48476,2914.30008,5852.25720,474.16635
2019-06-30,713.14600,190041.6084,127056.4758,NaN,448.86729,4957.31500,3413.42694,22854.73200,4156.04457,2580.34056,...,5078.80295,907.01305,183.62799,6019.71630,358.78395,NaN,1829.14176,1721.06535,4185.93744,507.38744
2020-06-30,625.85208,169606.0541,101480.2023,NaN,819.78780,5393.77056,NaN,22164.29040,3067.76880,2226.60900,...,4754.83140,802.73784,99.26686,5350.43642,NaN,NaN,NaN,312.41478,1626.83727,308.33152


In [17]:
S_cutoff = pd.to_numeric(me_nyse.quantile(.5, axis=1, numeric_only=False))
S_cutoff.head()

date
2016-06-30    2506.434960
2017-06-30    2886.386500
2018-06-30    3374.632515
2019-06-30    3176.082000
2020-06-30    2448.792270
Name: 0.5, dtype: float64

In [15]:
ME = df.unstack().xs('ME', axis=1)
ME = ME.loc[ME.index.month==6]
ME_port = pd.DataFrame(index=ME.index, columns=ME.columns)
ME.head()


PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,73.47888,411.47244,2220.568860,9.00017,1440.50400,58.94105,0.00000,168743.3389,399535.360,21.8929,...,248.67304,1685.40300,4226.75724,5405.28625,163.09326,1220.06808,5415.20670,32.5619,74.38596,31420.62420
2017-06-30,135.97100,NaN,2473.539217,44.93302,1765.45831,68.90184,NaN,207413.2355,531312.440,NaN,...,NaN,1911.33565,2429.09040,5401.26210,210.92160,1330.15890,10307.81780,NaN,74.61700,60339.32776
2018-06-30,NaN,NaN,2850.731590,19.38528,1926.77394,67.55320,NaN,175409.6893,757028.970,NaN,...,NaN,4115.48476,2914.30008,5852.25720,474.16635,NaN,11666.97549,NaN,84.47075,58478.46391
2019-06-30,NaN,NaN,3030.688500,35.00120,1721.15619,50.11440,713.14600,190041.6084,1023856.356,NaN,...,NaN,1829.14176,1721.06535,4185.93744,507.38744,NaN,11576.50730,NaN,87.86976,40025.71007
2020-06-30,NaN,NaN,2401.231440,164.23640,2059.36416,25.77625,625.85208,169606.0541,1540774.134,NaN,...,NaN,240.73800,312.41478,1626.83727,308.33152,NaN,10234.68160,NaN,76.22724,200844.67120


In [16]:
# compare dataframe and the cutoff

In [18]:
ME_port[ME.gt(S_cutoff, axis=0)] = 'Big' # gt: greater than
ME_port[ME.le(S_cutoff, axis=0)] = 'Small' # le: less then
ME_port.head()

PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,Small,Small,Small,Small,Small,Small,Small,Big,Big,Small,...,Small,Small,Big,Big,Small,Small,Big,Small,Small,Big
2017-06-30,Small,NaN,Small,Small,Small,Small,NaN,Big,Big,NaN,...,NaN,Small,Small,Big,Small,Small,Big,NaN,Small,Big
2018-06-30,NaN,NaN,Small,Small,Small,Small,NaN,Big,Big,NaN,...,NaN,Big,Small,Big,Small,NaN,Big,NaN,Small,Big
2019-06-30,NaN,NaN,Small,Small,Small,Small,Small,Big,Big,NaN,...,NaN,Small,Small,Big,Small,NaN,Big,NaN,Small,Big
2020-06-30,NaN,NaN,Small,Small,Small,Small,Small,Big,Big,NaN,...,NaN,Small,Small,Small,Small,NaN,Big,NaN,Small,Big


## Do the samething to Book to market ratio

In [19]:
bm_nyse = df_nyse.unstack().xs('BM', axis=1)
bm_nyse = bm_nyse.loc[bm_nyse.index.month==6]
bm_nyse.head()

PERMNO,10051,10104,10145,10147,10158,10220,10375,10516,10517,10606,...,93330,93372,93373,93374,93384,93418,93420,93422,93423,93426
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,NaN,0.319560,0.236042,0.424589,1.646971,0.078547,0.846846,0.889146,1.029988,0.555812,...,0.563636,0.541488,0.389621,0.740693,1.833255,0.708453,2.855260,2.291836,0.032601,1.255473
2017-06-30,NaN,0.300814,0.224878,NaN,1.980440,0.038080,0.646068,0.717326,0.769354,0.452688,...,0.448359,0.370780,0.732421,0.807990,0.606603,NaN,0.960353,0.981484,0.002317,0.748481
2018-06-30,NaN,0.277539,0.173783,NaN,1.425602,0.047463,0.680058,0.863985,0.691724,0.419240,...,0.349775,0.346567,0.803164,0.595677,0.423292,NaN,1.625491,1.871800,-0.071403,0.633590
2019-06-30,-0.031398,0.282542,0.203391,NaN,0.966414,0.062439,0.728538,0.872806,0.703160,0.518956,...,0.387541,0.406209,1.881732,0.793348,-2.522636,NaN,2.291180,2.265606,-0.099888,0.586222
2020-06-30,0.009219,0.129745,0.159434,NaN,0.844302,0.068341,NaN,0.790702,0.533931,0.369193,...,0.341019,0.287443,1.886844,0.715353,NaN,NaN,NaN,2.742920,-0.123003,0.576182


In [20]:
# 這次是分為 3 類
bm_nyse = bm_nyse.apply(pd.to_numeric)
L_cutoff = pd.to_numeric(bm_nyse.quantile(.3,axis=1,numeric_only=False))
H_cutoff = pd.to_numeric(bm_nyse.quantile(.7,axis=1,numeric_only=False))
H_cutoff.tail()

date
2016-06-30    0.797621
2017-06-30    0.655417
2018-06-30    0.639257
2019-06-30    0.827990
2020-06-30    0.747450
Name: 0.7, dtype: float64

In [21]:
BM = df.unstack().xs('BM', axis=1)
BM = BM.loc[BM.index.month==6]
BM_port = pd.DataFrame(index=BM.index, columns=BM.columns)
BM.head()

PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,1.377216,0.282954,0.295408,0.953993,0.733165,0.317002,0.284643,0.319560,0.188588,0.267774,...,0.708453,2.855260,2.291836,0.032601,1.255473,0.252853,0.049733,-0.837665,0.837790,0.034522
2017-06-30,0.789519,NaN,0.275043,0.384511,0.505577,0.309779,NaN,0.300814,0.152960,NaN,...,NaN,0.960353,0.981484,0.002317,0.748481,0.241305,0.052937,NaN,0.820741,0.137670
2018-06-30,NaN,NaN,0.262855,0.309202,0.502759,0.270789,NaN,0.277539,0.110646,NaN,...,NaN,1.625491,1.871800,-0.071403,0.633590,NaN,0.256208,NaN,0.645263,0.080625
2019-06-30,NaN,NaN,0.298914,0.679435,0.590952,0.385841,-0.031398,0.282542,0.106693,NaN,...,NaN,2.291180,2.265606,-0.099888,0.586222,NaN,0.335055,NaN,1.359421,0.085708
2020-06-30,NaN,NaN,0.257177,0.307973,0.387013,0.368183,0.009219,0.129745,0.085451,NaN,...,NaN,3.727508,2.742920,-0.123003,0.576182,NaN,0.282497,NaN,1.430444,0.087374


In [22]:
BM_port[BM.gt(H_cutoff, axis=0)] = 'Value'
BM_port[(BM.le(H_cutoff, axis=0)) & (BM.ge(L_cutoff, axis=0))] = 'Neutral'
BM_port[BM.lt(L_cutoff, axis=0)] = 'Growth'
BM_port.head()

PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-06-30,Value,Growth,Growth,Value,Neutral,Neutral,Growth,Neutral,Growth,Growth,...,Neutral,Value,Value,Growth,Value,Growth,Growth,Growth,Value,Growth
2017-06-30,Value,NaN,Growth,Neutral,Neutral,Neutral,NaN,Neutral,Growth,NaN,...,NaN,Value,Value,Growth,Value,Growth,Growth,NaN,Value,Growth
2018-06-30,NaN,NaN,Neutral,Neutral,Neutral,Neutral,NaN,Neutral,Growth,NaN,...,NaN,Value,Value,Growth,Neutral,NaN,Neutral,NaN,Value,Growth
2019-06-30,NaN,NaN,Growth,Neutral,Neutral,Neutral,Growth,Growth,Growth,NaN,...,NaN,Value,Value,Growth,Neutral,NaN,Neutral,NaN,Value,Growth
2020-06-30,NaN,NaN,Growth,Neutral,Neutral,Neutral,Growth,Growth,Growth,NaN,...,NaN,Value,Value,Growth,Neutral,NaN,Neutral,NaN,Value,Growth


## Get the return for all stocks

In [23]:
RET = df.unstack().xs('RET', axis=1)
RET = RET.apply(pd.to_numeric)
RET.tail()

PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2020-08-31,NaN,NaN,0.104118,-0.082742,0.023960,-0.018072,0.131730,0.031921,0.102580,NaN,...,NaN,-0.132187,-0.115646,0.249569,-0.021218,NaN,0.051425,NaN,0.008584,0.741452
2020-09-30,NaN,NaN,-0.036668,0.105670,-0.071513,-0.177914,-0.199393,0.043341,-0.067397,NaN,...,NaN,-0.495859,-0.305538,-0.065808,0.016459,NaN,-0.044122,NaN,0.055319,-0.139087
2020-10-31,NaN,NaN,0.039727,-0.058275,-0.015432,0.000000,0.104298,-0.056114,-0.037370,NaN,...,NaN,NaN,-0.003102,0.065025,-0.056477,NaN,-0.073513,NaN,-0.080645,-0.095499
2020-11-30,NaN,NaN,0.072435,0.143564,0.074346,0.593284,0.298798,0.028694,0.060058,NaN,...,NaN,NaN,0.788889,0.421369,0.224362,NaN,0.128552,NaN,0.144737,0.462736
2020-12-31,NaN,NaN,0.072598,0.125541,0.046848,-0.051522,-0.030851,0.120755,0.039006,NaN,...,NaN,NaN,0.484472,0.109665,0.076239,NaN,0.019711,NaN,0.122605,0.243252


## Construct 2 by 3 ME-BM model (Unconditional Bivariate sort)

In [25]:
TMP2 = ME_port + BM_port # only contains data in June
ME_BM_port2 = pd.DataFrame(index=RET.index) # the dataframe after bivariate sort 
ME_BM_port2 = ME_BM_port2.join(TMP2)

In [30]:
ME_BM_port2.head(7)

,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-30,SmallValue,SmallGrowth,SmallGrowth,SmallValue,SmallNeutral,SmallNeutral,SmallGrowth,BigNeutral,BigGrowth,SmallGrowth,...,SmallNeutral,SmallValue,BigValue,BigGrowth,SmallValue,SmallGrowth,BigGrowth,SmallGrowth,SmallValue,BigGrowth
2016-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
TMP = ME_port + BM_port # only contains data in June
ME_BM_port = pd.DataFrame(index=RET.index) # the dataframe after bivariate sort 
ME_BM_port = ME_BM_port.join(TMP)
ME_BM_port = ME_BM_port.ffill(axis=0,limit=11).shift(1) # ffill = fill forward
# shift: bc we know the situation on June, but we modify on July, so we need to shift for one month
ME_BM_port.tail()

,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2020-08-31,NaN,NaN,SmallGrowth,SmallNeutral,SmallNeutral,SmallNeutral,SmallGrowth,BigGrowth,BigGrowth,NaN,...,NaN,SmallValue,SmallValue,SmallGrowth,SmallNeutral,NaN,BigNeutral,NaN,SmallValue,BigGrowth
2020-09-30,NaN,NaN,SmallGrowth,SmallNeutral,SmallNeutral,SmallNeutral,SmallGrowth,BigGrowth,BigGrowth,NaN,...,NaN,SmallValue,SmallValue,SmallGrowth,SmallNeutral,NaN,BigNeutral,NaN,SmallValue,BigGrowth
2020-10-31,NaN,NaN,SmallGrowth,SmallNeutral,SmallNeutral,SmallNeutral,SmallGrowth,BigGrowth,BigGrowth,NaN,...,NaN,SmallValue,SmallValue,SmallGrowth,SmallNeutral,NaN,BigNeutral,NaN,SmallValue,BigGrowth
2020-11-30,NaN,NaN,SmallGrowth,SmallNeutral,SmallNeutral,SmallNeutral,SmallGrowth,BigGrowth,BigGrowth,NaN,...,NaN,SmallValue,SmallValue,SmallGrowth,SmallNeutral,NaN,BigNeutral,NaN,SmallValue,BigGrowth
2020-12-31,NaN,NaN,SmallGrowth,SmallNeutral,SmallNeutral,SmallNeutral,SmallGrowth,BigGrowth,BigGrowth,NaN,...,NaN,SmallValue,SmallValue,SmallGrowth,SmallNeutral,NaN,BigNeutral,NaN,SmallValue,BigGrowth


In [34]:
ME_BM_port.head(7)

,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-31,SmallValue,SmallGrowth,SmallGrowth,SmallValue,SmallNeutral,SmallNeutral,SmallGrowth,BigNeutral,BigGrowth,SmallGrowth,...,SmallNeutral,SmallValue,BigValue,BigGrowth,SmallValue,SmallGrowth,BigGrowth,SmallGrowth,SmallValue,BigGrowth


In [36]:
df.unstack().xs('ME', axis=1).shift(1)

PERMNO,10001,10025,10026,10028,10032,10044,10051,10104,10107,10116,...,93418,93420,93422,93423,93426,93428,93429,93433,93434,93436
date,,,,,,,,,,,,,,,,,,,,,
2016-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-29,87.40160,432.25648,2017.714280,4.918400,1162.99620,61.075940,476.07559,152546.2982,4.357235e+05,21.831850,...,167.45369,757.164100,2265.768340,4616.64599,143.54208,994.82680,5468.76918,32.921700,60.74622,25249.10720
2016-03-31,82.56930,401.97197,2070.221940,7.377600,1214.51625,60.200090,0.00000,154520.8716,4.024253e+05,25.561440,...,206.45160,973.202230,1725.148320,4657.65708,142.32768,1074.42125,5112.18750,38.273725,70.45066,25345.50676
2016-04-30,82.06748,337.06200,2015.848760,6.026020,1318.98000,59.382630,0.00000,169770.9772,4.346601e+05,33.490220,...,188.77480,1314.294800,2959.671270,5170.28075,170.26353,1180.96380,5327.72683,35.440300,70.28670,30756.55266
2016-05-31,76.73760,315.17582,1882.939470,6.735615,1389.22992,58.798730,0.00000,165413.6189,3.920015e+05,38.815520,...,182.98120,1749.384150,3886.291640,5601.16375,181.80888,1148.16114,5043.29616,31.842300,77.18085,32248.59686
2016-06-30,75.05568,306.89114,1964.304500,6.534370,1464.51240,58.214830,0.00000,166824.5730,4.166048e+05,29.821680,...,182.97664,1811.718000,4038.015240,5381.03475,164.18703,1278.17760,5180.85540,30.133250,76.60380,32936.02389
2016-07-31,73.47888,411.47244,2220.568860,9.000170,1440.50400,58.941050,0.00000,168743.3389,3.995354e+05,21.892900,...,248.67304,1685.403000,4226.757240,5405.28625,163.09326,1220.06808,5415.20670,32.561900,74.38596,31420.62420
2016-08-31,74.63520,411.47244,2268.999380,10.479650,1535.36074,61.612270,0.00000,169196.8392,4.416798e+05,22.980445,...,249.70788,1371.420000,4360.829200,5238.57461,159.81195,1336.33747,5592.40800,27.164900,81.72219,34911.62798
2016-09-30,77.39776,565.60840,2276.276000,10.505150,1542.93295,64.101300,0.00000,169938.9306,4.477579e+05,18.934400,...,0.00000,1710.485880,4576.474600,4530.68423,185.79009,1362.56939,5583.46665,26.985000,81.89280,31640.15890


In [38]:
ME_lag = df.unstack().xs('ME', axis=1).shift(1)

unique_port = ['SmallGrowth', 'SmallNeutral', 'SmallValue', 'BigGrowth', 'BigNeutral', 'BigValue']
# empty df for storing rturns for the 6 stragtegies
RET_port = pd.DataFrame(index=RET.index, columns=unique_port)

# empty df fo sotring the noum of stocks in each strategy
N_firm = pd.DataFrame(index=RET.index, columns=unique_port)

In [39]:
for p in unique_port:
    TMP_RET = RET[ME_BM_port==p].apply(pd.to_numeric)
    TMP_ME = ME_lag[ME_BM_port==p].apply(pd.to_numeric)
    TMP_PROD = TMP_RET*TMP_ME
    RET_port[p] = TMP_PROD.sum(axis=1)/TMP_ME.sum(axis=1)
    # calculate the value weighted return 
    N_firm[p] = TMP_RET.count(axis=1)
RET_port = RET_port.dropna()*100 # for the first siz month, we don't have portfolios
# 第一筆資料從 2016.06 開始
RET_port.tail()

,SmallGrowth,SmallNeutral,SmallValue,BigGrowth,BigNeutral,BigValue
date,,,,,,
2020-08-31,6.233505,4.766238,6.298374,9.534417,3.457365,4.058304
2020-09-30,-1.913589,-4.292057,-5.627923,-4.058011,-3.003876,-4.335978
2020-10-31,0.820848,2.704116,5.279059,-3.454844,-0.370999,0.522830
2020-11-30,23.829603,17.713952,20.053387,10.705744,13.794444,16.566307
2020-12-31,12.861766,8.273868,8.021720,4.621791,4.229645,4.867752
